In [1]:
import json
import requests
from bs4 import BeautifulSoup
from langdetect import detect

In [2]:
base_url = 'https://rozetka.com.ua/ua/mobile-phones/c80003/' #mobile

In [3]:
def download_html(link: str, page: int = 1):
    response = requests.get(link+'page='+str(page))
    return BeautifulSoup(response.text, 'html.parser')

def get_number_of_pages(soup):
    pages = soup.find_all('span', class_ = 'paginator-catalog-l-i-active')
    if pages:
        return int(pages[-1].text)
    else:
        return 1

In [4]:
all_links = set()

def find_links(url: str, page: int = 1):
    global all_links
    soup = download_html(url, page)
    for t in soup.find_all('div', class_ = 'g-i-tile-i-title'):
        all_links.add(t.a.get('href'))
    return soup
    
soup = find_links(base_url)
pages = get_number_of_pages(soup)
for page in range(2, pages + 1):
    print(f'\rPage {page} from {pages}', end='')
    find_links(base_url, page)

Page 100 from 100

In [5]:
with open('rozetka_mobile_index.txt', 'w+') as fw:
    for l in all_links:
        fw.write(l+'\n')

In [6]:
uk_reviews = []

def find_reviews(url: str, page: int = 1):
    "find all reviews on a page"
    
    global uk_reviews
    
    soup = download_html(url + 'comments/', page)
    reviews = soup.find_all('div', {'itemprop': 'review'})
    for r in reviews:
        rating = r.find('span', class_ = 'g-rating-stars-i')
        if rating:
            rating = int(rating.get('content'))

        comment = r.find('div', class_ = 'pp-review-text')
        if comment:
            comment = comment.text\
            .strip()\
            .replace('\n\n\n', '\n')\
            .replace('Переваги:', '+')\
            .replace('Недоліки:', '-')

        if rating and len(comment) > 3:
            try:
                if detect(comment) == 'uk':
                    uk_reviews.append({'Comment': comment, 'Rating': rating})
            except: pass
    return soup

In [7]:
for i, link in enumerate(all_links):
    print(f'\rItem {i} / {len(all_links)}, page 1 / {pages}, reviews: {len(uk_reviews)}', end = '')
    soup = find_reviews(link)
    pages = get_number_of_pages(soup)
    
    for page in range(2, pages + 1):
        print(f'\rItem {i} / {len(all_links)}, page {page} / {pages}, reviews: {len(uk_reviews)}', end = '')
        find_reviews(link, page)

Item 3011 / 3012, page 10 / 10, reviews: 6025

In [8]:
with open('rozetka_mobile_reviews.json', 'w+') as fw:
    json.dump(uk_reviews, fw, ensure_ascii=False)